#### In this notebook, we demonstrate how to use the DLN code to train, evaluate, and visualize models.

In [1]:
# cd to the parent directory
import os
os.chdir("./..")

import json
import csv
from experiments.utils import *

#### Training

#### Let's use the dataset we prepared in the prepare_dataset notebook. We present a general use case here. For more advanced functions such as pruning, freezing, and the unified phase, see the descriptions in `experiments/main.py`.

In [ ]:
!python experiments/main.py \
--train_model True \
--dataset Heart \
--seed 0 \
--num_epochs 1000 \
--batch_size 64 \
--learning_rate 0.2 \
--tau_out 3 \
--grad_factor 1.2 \
--first_hl_size 50 \
--last_hl_size_wrt_first 0.25 \
--num_hidden_layers 4 \
--num_heads 1 \
--discretize_strategy tree \
--continuous_resolution 4 \
--concat_input True \
--save_model

# last_hidden_layer_size = first_hidden_layer_size x last_hl_size_wrt_first
# The middle hidden layers will have sizes in a geometric progression from the first to the last layer
# Will save the model with the best mean train + val balanced-class accuracy

#### Evaluation

#### We can use the ```--evaluate_model``` flag, which loads the model and evaluates its balanced-class accuracy. It then attempts to simplify the model using SymPy before evaluating the model’s high-level OPs, basic gate-level OPs, number of parameters, and disk space usage. If simplification is successful, the simplified model is used for these evaluations.

In [ ]:
!python experiments/main.py \
--train_model False \
--evaluate_model True \
--dataset Heart \
--seed 0

In [ ]:
# Read the eval results

import json
from experiments.utils import *

results_path = get_results_path(dataset='Heart', seed=0)
with open(f"{results_path}/eval_results.json", 'r') as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

#### Evaluation Loop

In [ ]:
head_vals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
epochs = 4000

results_path = get_results_path(dataset='Heart', seed=0)
file_path = f"{results_path}/results_v3_{epochs}.txt"
csv_file_path = file_path.replace(".txt", ".csv")

# Prepare the CSV file with a header
with open(csv_file_path, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["num_heads", "acc_testLoader_eval_mode"])

for val in head_vals:
    !python experiments/main.py \
    --train_model True \
    --dataset Heart \
    --seed 0 \
    --num_epochs {epochs} \
    --batch_size 64 \
    --learning_rate 0.2 \
    --tau_out 3 \
    --grad_factor 1.2 \
    --first_hl_size 50 \
    --last_hl_size_wrt_first 0.25 \
    --num_hidden_layers 4 \
    --num_heads {val} \
    --discretize_strategy tree \
    --continuous_resolution 4 \
    --concat_input True \
    --save_model

    !python experiments/main.py \
    --train_model False \
    --evaluate_model True \
    --dataset Heart \
    --seed 0

    with open(f"{results_path}/eval_results.json", 'r') as f:
        data = json.load(f)

    # Extract the accuracy value
    acc_eval_mode = data.get("acc_testLoader_eval_mode", None)
    if acc_eval_mode is not None:
        # Append the num_heads and accuracy to the CSV file
        with open(csv_file_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow([val, acc_eval_mode])

    results_string = json.dumps(data, indent=4)
    with open(file_path, "a") as file:
        file.write(f"num_heads = {val}:\n" + results_string + '\n\n')

len(head_train_loaders): 1
NUM_HEADS: 1
----------
depth: 7; num layers: 10
idx: 0; layer: ThresholdLayer(34, 34), depth: 1, next_layer_idx: 1
idx: 1; layer: LogicLayer(34, 50), depth: 2, next_layer_idx: 2
idx: 2; layer: LogicLayer(84, 31), depth: 3, next_layer_idx: 3
idx: 3; layer: LogicLayer(65, 19), depth: 4, next_layer_idx: 4
idx: 4; layer: LogicLayer(53, 12), depth: 5, next_layer_idx: 5
idx: 5; layer: SumLayer(
  12, 2
  (sum_weights): ParameterList(  (0): Parameter containing: [torch.float32 of size 12x2])
), depth: 6, next_layer_idx: 6
idx: 6; layer: MultiHeadedSumLayer(
  12, 2
  (sum_weights): ParameterList(  (0): Parameter containing: [torch.float32 of size 12x2])
), depth: 7, next_layer_idx: None
idx: 7; layer: ThresholdLayer(34, 34), depth: 2, next_layer_idx: 2
idx: 8; layer: ThresholdLayer(34, 34), depth: 3, next_layer_idx: 3
idx: 9; layer: ThresholdLayer(34, 34), depth: 4, next_layer_idx: 4
----------


#### Visualization

#### We use Graphviz to render DLNs generated from SymPy code.

In [ ]:
!python experiments/DLN_viz.py \
results/Heart/seed_0/sympy_code.py \
quickstart/example/viz

# A file named viz.png will be created

Traceback (most recent call last):
  File "/Users/ash/Projects/apazhetam/dln/experiments/DLN_viz.py", line 189, in <module>
    dicts = execute_and_capture_dicts(args.sympy_code_path)
  File "/Users/ash/Projects/apazhetam/dln/experiments/DLN_viz.py", line 175, in execute_and_capture_dicts
    raise ValueError("Script did not produce any output.")
ValueError: Script did not produce any output.


In [ ]:
from IPython.display import Image

Image("quickstart/example/viz.png")

#### The Heart dataset has 5 continuous and 14 categorical features. The DLN uses only 2 continuous and 11 categorical features.